In [5]:
import pandas as pd
import numpy as np

In [179]:
mail_greg = 'gregoire.clement@epfl.ch'

In [180]:
column_names = ['mail', 'movie', 'date', 'rating']

crypt_cols = column_names[:2]
clear_cols = column_names[2:]

In [352]:
fnames_clear = [f'imdb-{i+1}.csv' for i in range(3)]

data_clear = []  

for fname in fnames_clear:
    with open(f'{fname[:-4]}.csv', 'r') as file:
        lines_cleaned = []
        lines = file.readlines()
        for i, line in enumerate(lines):
            plus = 1 if len(lines) == i+1 else 0
            line_cleaned = line[:-4+plus]\
                .replace('", ', '", "')\
                .replace('ch, ', 'ch", ')\
                .replace('com, ', 'com", ')\
                + '", "' + line[-2+plus]
            line_cleaned = line_cleaned.split('", "')
            assert len(line_cleaned) == 4
            lines_cleaned.append(line_cleaned)
    df = pd.DataFrame(lines_cleaned, columns=column_names)
    data_clear.append(df)


In [373]:
data_crypt = [pd.read_csv(f'com402-{i+1}.csv', names=column_names) for i in range(3)]

def clean(df):
    for c in column_names:
        df[c] = df[c].apply(lambda x: str(x).strip().strip('"'))
    df['rating'] = df['rating'].apply(lambda x: int(x))
    df['date'] = df['date'].apply(lambda x: int(pd.to_datetime(x, format='%d/%m/%y').timestamp())//(60*60*24))
    return df
    
for i in range(3):
    data_crypt[i] = clean(data_crypt[i])
    data_clear[i] = clean(data_clear[i])

In [374]:
set(data_crypt[0]['rating']), set(data_clear[0]['rating'])

({1, 2, 3, 4, 5}, {1, 2, 3, 4, 5})

In [375]:
data_crypt[0].head()

,mail,movie,date,rating
0,baacb66ef94dd3ee99e1c6d2c09162a91fedb3728d3d88...,44ff990dae6198226ec564d0f929d74c68941556925e51...,11688,3
1,d573cb5bb85f9e804319e73365cdace936e0068763ccfa...,77759aba50c2cd39076b2e6f1afd040875e64192e80588...,11688,2
2,814a6e672abe1cf98d0683c011fd5215d3eeb73c983a8c...,3e0255de2b47d08b88dfb2b31034d7a2d0c276ed1ff0dd...,13514,2
3,328d393a6d782839d3845786684a9d6b750e6d1fd24842...,928fa3ed8d0d3de5e10f0253f295c544307a5f29fa4429...,14610,4
4,112981ec2c0db94251d9cdca1c59a90ce5e369a78b1249...,73a4e2545a68c493e65e9a49808a950573ed6225975509...,11719,5


# Part 1

In [376]:
set(data_crypt[0]['rating'])

{1, 2, 3, 4, 5}

In [377]:
my_ratings_1 = data_clear[0].set_index('mail').loc[mail_greg][clear_cols].values.tolist()
my_ratings_1

[[15928, 3], [14986, 1], [16598, 1], [11132, 4], [13116, 3]]

In [378]:
my_ratings_1[0]

[15928, 3]

In [379]:
data_clear[0].set_index(clear_cols).loc[my_ratings_1[0]]

,,mail,movie
date,rating,,
15928,3,gregoire.clement@epfl.ch,Goodfellas


In [380]:
data_crypt[0].set_index(clear_cols).loc[my_ratings_1[0]]

,,mail,movie
date,rating,,
15928,3,0a9577eb6c5aec7d2755674930a6d0be4343ae8cd22f88...,f3e479ad5c430c1114814170bfdffb91f261dac75c1123...


## Just to be sure

In [381]:
associated_crypt = [data_crypt[0].set_index(clear_cols).loc[r].values.tolist() for r in my_ratings_1]
possible_crypt_mail = {m:0 for l in associated_crypt for m, _ in l}

for l in associated_crypt:
    for m, _ in l:
        possible_crypt_mail[m] += 1
        
        
my_mail_crypt_1 = max(possible_crypt_mail.items(), key=lambda x: x[1])[0]
my_mail_crypt_1

'0a9577eb6c5aec7d2755674930a6d0be4343ae8cd22f886104703e1fd536a324'

In [382]:
my_movies_crypt = data_crypt[0].set_index(['mail']).loc[my_mail_crypt_1]['movie'].values.tolist()

In [383]:
count_per_movie = {m:dict() for m in my_movies_crypt}
movie_to_name = dict()

for m in my_movies_crypt:
    ratings = data_crypt[0].set_index('movie').loc[m][clear_cols].values.tolist()
    for r in ratings:
        for movie_name in data_clear[0].set_index(clear_cols).loc[r]['movie'].values.tolist():
            count_per_movie[m][movie_name] = count_per_movie[m].get(movie_name, 0) + 1
    movie_to_name[m] = max(count_per_movie[m].items(), key=lambda x: x[1])[0]

In [384]:
movie_to_name

{'c7e94fe3900cc5a4013b912b29b8e668febedc18ebc3b790d8f2d0a81ed6357c': 'The Night of the Hunter',
 '65b3ace60011c8dcacef9e0ba9e85eca78baa76a817360da584c147330a27845': 'The 400 Blows',
 'f3e479ad5c430c1114814170bfdffb91f261dac75c112323e2dba42b51d198ee': 'Goodfellas',
 '61d50c589d14df9becd1b630dfd5caba0bdad6771415a282a33c4f0df803a5b7': 'Fanny and Alexander',
 '68d84c3e5d6648fe935bf8922dbdda79fb09359854a02a8a0998cf90bb04f26e': 'Paths of Glory',
 '6f0aed3bf9fae6199d41ce5fbb1ba0cfae6994296668e45cea979e5976b42757': 'Once Upon a Time in the West',
 'a56a713016a3947059840712435afa5a77475691918448a6f4daf49d81b39d3c': 'Spirited Away',
 '2356e206c533e3a35e7c8248d7f5a3f4a10caa0e545103c064ec62074287bf66': 'The Godfather: Part II',
 'e218ba02398a45d0c18232b2f8878e27815746861dbabee215f6fc4499cc9c2c': 'Rear Window',
 '78ad872070f45601d154c0b4c561b6c25a5a2ad602083cef774b592870c9aade': 'The Third Man'}

In [388]:
with open('ex1a.txt', 'w') as file: 
    file.write('\n'.join(movie_to_name.values()))

# Part 2

In [389]:
set(data_clear[1]['rating'].values)

{1, 2, 3, 4, 5}

In [390]:
len(data_clear[1]), len(data_crypt[1])

(4554, 5060)

In [391]:
data_clear[1].set_index('mail').loc[mail_greg]['movie'].values.tolist()

['Seven Samurai',
 'Psycho',
 "Singin' in the Rain",
 'The Third Man',
 'The Good, the Bad and the Ugly',
 'M',
 'The Godfather',
 'Lawrence of Arabia',
 '12 Angry Men',
 'Spirited Away',
 "Schindler's List",
 'City Lights',
 'Bicycle Thieves',
 'Goodfellas',
 'Pulp Fiction',
 'Apocalypse Now',
 'The Shawshank Redemption',
 'Metropolis']

In [392]:
movie_crypt_freq = data_crypt[1]\
    .pivot_table(values='date', index='movie', aggfunc=len)\
    .fillna(0)\
    .sort_values('date')\
    .reset_index()
movie_crypt_freq.head()

,movie,date
0,1950dcc430ede76265e70d2add492dba8b8f92b3a5a919...,1
1,1ff7ca975bb4e22646fb68093b279a1d904717c99cd8c0...,15
2,497d48a05bedb560fef6604ac6aaa1da192cc0e65e3dd9...,16
3,3f4ee2f4cf78ba35a818970f17d074f864b8145a742ab7...,22
4,176a3a37ec973f5322bd893fabfbd6f4ab40d78f04069d...,29


In [393]:
movie_clear_freq = data_clear[1]\
    .pivot_table(values='date', index='movie', aggfunc=len)\
    .fillna(0)\
    .sort_values('date')\
    .reset_index()
movie_clear_freq.head()

,movie,date
0,A Separation,1
1,The Silence of the Lambs,5
2,All About Eve,6
3,Tokyo Story,11
4,The Wages of Fear,19


In [394]:
sha_to_movie = dict(pd.concat((movie_crypt_freq, movie_clear_freq), axis=1)['movie'].values.tolist())
movie_to_sha = {m:s for s,m in sha_to_movie.items()}

In [395]:
movie_to_sha['A Separation']

'1950dcc430ede76265e70d2add492dba8b8f92b3a5a9191c05345a3b4e782f9f'

In [397]:
my_movies_2 = data_clear[1].set_index('mail').loc[mail_greg]['movie'].values.tolist()
len(my_movies_2)

18

In [398]:
my_movies_2

['Seven Samurai',
 'Psycho',
 "Singin' in the Rain",
 'The Third Man',
 'The Good, the Bad and the Ugly',
 'M',
 'The Godfather',
 'Lawrence of Arabia',
 '12 Angry Men',
 'Spirited Away',
 "Schindler's List",
 'City Lights',
 'Bicycle Thieves',
 'Goodfellas',
 'Pulp Fiction',
 'Apocalypse Now',
 'The Shawshank Redemption',
 'Metropolis']

In [399]:
all_mails = []
for m in my_movies_2:
    mails = data_crypt[1].set_index('movie').loc[movie_to_sha[m]]['mail']
    if type(mails) == str:
        all_mails.append(mails)
    else:
        for mail in mails.values.tolist():
            all_mails.append(mail)

In [443]:
all_mails = [mail for m in my_movies_2 for mail in data_crypt[1].set_index('movie').loc[[movie_to_sha[m]]]['mail'].values.tolist()]

In [445]:
sorted({m:all_mails.count(m) for m in set(all_mails)}.items(), key=lambda x:x[1], reverse=True)[:5]

[('0c602ad9789dad6442f6d2d54a199c3b69258477457efe22c9baef2c5010a180', 18),
 ('a1ae87cd9a492c0d50ddf701c5884fb9fbd2b16c98d4ff6d79c05bdd63b4899f', 14),
 ('c30e5c041d2452f3981ea3a22cac42b8a736bf7ffcfcf4baa46eca3a87ebb19d', 14),
 ('4f4bc4054e14c5291242d2accc17489b895328742e96339b3326dfa957c120b6', 14),
 ('3e066b8cb1acf0412ae2e621f66e32ba79e0b78b5359ba4230498f5c65462308', 13)]

In [401]:
mail_crypt_2, _ = max({m:all_mails.count(m) for m in set(all_mails)}.items(), key=lambda x:x[1])

In [402]:
my_movies_crypt_2 = data_crypt[1].set_index(['mail']).loc[mail_crypt_2]['movie'].values.tolist()

In [403]:
movie_to_name_2 = {m:sha_to_movie[m] for m in my_movies_crypt_2}

In [404]:
set(movie_to_name_2.values()) - set(my_movies_2)

{'On the Waterfront', 'Raiders of the Lost Ark'}

In [405]:
with open('ex1b.txt', 'w') as file:
    txt = '\n'.join(movie_to_name_2.values())
    print(txt)
    file.write(txt)

Psycho
Seven Samurai
Apocalypse Now
Metropolis
Spirited Away
The Shawshank Redemption
Pulp Fiction
City Lights
12 Angry Men
The Godfather
Singin' in the Rain
The Good, the Bad and the Ugly
Bicycle Thieves
Lawrence of Arabia
On the Waterfront
Goodfellas
M
The Third Man
Schindler's List
Raiders of the Lost Ark


# Part 3

In [406]:
len(data_clear[2]), len(data_crypt[2])

(12650, 25300)

In [407]:
weights = list(range(1,15))+list(range(13, 0, -1))

In [408]:
data_clear[2].head()

,mail,movie,date,rating
0,taehyun.lee@epfl.ch,Badlands,11323,4
1,valentin.meylan@epfl.ch,Boyhood,11323,2
2,delphine.peter@epfl.ch,Memories of Murder,11323,1
3,irina.kozlova@epfl.ch,L'Avventura,11688,1
4,mohamed.ndoye@epfl.ch,M,11688,4


In [409]:
a = pd.to_datetime('01/02/03', format='%d/%m/%y')

In [319]:
set_my_movies = list(set(data_clear[2].set_index('mail').loc[mail_greg]['movie'].values.tolist()))
set_movies_clear = list(set(data_clear[2]['movie'].values.tolist()))
set_movies_crypt = list(set(data_crypt[2]['movie'].values.tolist()))

In [452]:
data_clear[2]['date'].head()

0    11323
1    11323
2    11323
3    11688
4    11688
Name: date, dtype: int64

In [453]:
from tqdm import tqdm_notebook as tqdm

In [454]:
movie_to_sha = dict()
for movie in tqdm(set_movies_clear):
    movie_score = dict()
    for date in data_clear[2].set_index('movie').loc[movie]['date'].values.tolist():
        mask = data_crypt[2]['date'].apply(lambda x: date - 14 < x < date + 14)
        for movie_crypt, date2 in data_crypt[2][mask][['movie', 'date']].values.tolist():
            movie_score[movie_crypt] = movie_score.get(movie_crypt, 0) + weights[date - date2 + 13]
    movie_to_sha[movie] = max(movie_score.items(), key=lambda x:x[1])[0]

In [459]:
sha_to_movie = {s:m for m,s in movie_to_sha.items()}

In [461]:
len(movie_to_sha), len(sha_to_movie)

(300, 300)

In [466]:
all_mails = []
for m in set_my_movies:
    mails = data_crypt[2].set_index('movie').loc[movie_to_sha[m]]['mail']
    if type(mails) == str:
        all_mails.append(mails)
    else:
        for mail in mails.values.tolist():
            all_mails.append(mail)

In [467]:
sorted({m:all_mails.count(m) for m in set(all_mails)}.items(), key=lambda x:x[1], reverse=True)[:5]

[('f189cc88622d2f85b27c86fcec9a644bd21e34f8675757d668622dd173d436d3', 50),
 ('5d113001f2951ad22282fa5acf63955c8c8f8da4a56cea1e56b2d8e54f2a901c', 26),
 ('d44ec2cf4bc620726afa6799b6ff8e083cfb30b6d123e87ce1718b6e60cbc7a1', 26),
 ('f700431f8905508072db828ff88d1d61a969502f72987e3ce5598abb57f8a95b', 24),
 ('e90eceb959daeb4334baa6e2d0952f1011c9a64db48f8a8ff38ea3309614925b', 24)]

In [469]:
my_mail_crypt_3, _ = max({m:all_mails.count(m) for m in set(all_mails)}.items(), key=lambda x:x[1])
my_mail_crypt_3

'f189cc88622d2f85b27c86fcec9a644bd21e34f8675757d668622dd173d436d3'

In [471]:
my_movies_crypt_3 = data_crypt[2].set_index(['mail']).loc[my_mail_crypt_3]['movie'].values.tolist()

In [474]:
my_movies_3 = [sha_to_movie[m] for m in my_movies_crypt_3]

In [475]:
with open('ex1c.txt', 'w') as file:
    txt = '\n'.join(my_movies_3)
    print(txt)
    file.write(txt)

Full Metal Jacket
Once Upon a Time in America
The Iron Giant
A Streetcar Named Desire
Hannah and Her Sisters
Dog Day Afternoon
Ran
The Matrix
Throne of Blood
Toy Story 3
Spirited Away
The Exterminating Angel
The Shawshank Redemption
The Lord of the Rings: The Return of the King
Harold and Maude
The Wages of Fear
Harry Potter and the Deathly Hallows: Part 2
Rosemary's Baby
Blue Velvet
Terminator 2: Judgment Day
In the Mood for Love
Kind Hearts and Coronets
The Great Escape
Frankenstein
The Seventh Seal
Toy Story 2
The Philadelphia Story
The Gold Rush
Lagaan: Once Upon a Time in India
To Be or Not to Be
To Kill a Mockingbird
Blade Runner
Yojimbo
Star Wars: Episode V - The Empire Strikes Back
Saving Private Ryan
Whiplash
Incendies
Rome, Open City
Laura
Cinema Paradiso
Life of Brian
Before Sunset
Persepolis
Trainspotting
The Diving Bell and the Butterfly
It's a Wonderful Life
The Deer Hunter
The Last Picture Show
Fight Club
Citizen Kane
Woman in the Dunes
L.A. Confidential
Anatomy of a Mur